# Testing Metrics

## Testing Starters

In [1]:
import pandas as pd
import cv2
import os
from math import sqrt
from statistics import median, mean, stdev
import json

data_amount = 500

trainPath = f"train_annotations"
train_data =pd.read_json(trainPath)
trainTestOutputPath = "trainTestOutput"

input_directory_path = "processed_foregrounds"
output_directory_path = "detection_v2_final"

if os.path.exists(output_directory_path) and os.path.isdir(output_directory_path):
    pass
else: 
    os.mkdir(output_directory_path)

## Testing Functions

In [2]:
def calculateOverlapArea(box1, box2):
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x1_intersection = max(x1_box1, x1_box2)
    y1_intersection = max(y1_box1, y1_box2)
    x2_intersection = min(x2_box1, x2_box2)
    y2_intersection = min(y2_box1, y2_box2)

    # Check if there is no intersection (negative width or height)
    if x1_intersection >= x2_intersection or y1_intersection >= y2_intersection:
        return 0

    # Calculate width and height of the intersection
    width_intersection = x2_intersection - x1_intersection + 1
    height_intersection = y2_intersection - y1_intersection + 1

    # Calculate the overlapping area
    overlapping_area = width_intersection * height_intersection

    return overlapping_area

def getContours(image):
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, thresholded = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY)
    
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # print(f"Type of contours: {type(contours)}")
    return contours

# All areas
def getBoundingBox(contours): 
    x_min = x_max = 0
    y_min = y_max = 0
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if (x_min == 0) and (x_max == 0) and (y_min == 0) and (y_max == 0):
            x_min = x
            x_max = x + w
            y_min = y
            y_max = y + h
        if (x < x_min): x_min = x
        elif (x + w > x_max): x_max = x + w
        if (y < y_min): y_min = y
        elif (y + h > y_max): y_max = y + h
        
    return (x_min, y_min), (x_max, y_max)

def drawBoundingBox(original_image, top_left_corner: tuple, bottom_right_corner: tuple, colour: tuple):
    drawn = cv2.rectangle(original_image, top_left_corner, bottom_right_corner, colour, 2)
    return drawn

## Driving Code

In [40]:
centreDeltas = {}
overlappingAreas = []
passed = 0
failed = 0
for i in range(data_amount):
    islandImagePath = input_directory_path + f"/processed_result{i:03d}.png"
    island_image = cv2.imread(islandImagePath)
    
    originalImagePath = f"train/image_id_{i:03d}.jpg"
    original_image = cv2.imread(originalImagePath)
    
    # Get contours
    contours = getContours(island_image)

    # Get the coordinates of the bounding box to be drawn
    top_left_corner, bottom_right_corner = getBoundingBox(contours)
        
    # True info
    true_x, true_y, true_w, true_h = tuple(train_data["bbox"][i])
    true_centre_x = int(true_x + (true_w / 2))
    true_centre_y = int(true_y + (true_h / 2))
    true_centre = (true_centre_x, true_centre_y)

    # Predicted info
    predicted_w = bottom_right_corner[0] - top_left_corner[0]
    predicted_h = bottom_right_corner[1] - top_left_corner[1]
    predicted_centre_x = int(top_left_corner[0] + (predicted_w / 2))
    predicted_centre_y = int(top_left_corner[1] + (predicted_h / 2))
    predicted_centre = (predicted_centre_x, predicted_centre_y)

    # Add prediction box
    green = (0, 255, 0)
    drawn_image = drawBoundingBox(original_image, top_left_corner=top_left_corner, bottom_right_corner=bottom_right_corner, colour=green)
    cv2.drawMarker(drawn_image, predicted_centre, green, markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
    
    # Add true box
    red = (0, 0, 255)
    drawn_image = drawBoundingBox(drawn_image, top_left_corner=(true_x, true_y), bottom_right_corner=((true_x + true_w - 1), true_y + true_h - 1), colour=red)
    cv2.drawMarker(drawn_image, true_centre, red, markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)
    
    # Calculate distances
    centre_distance_delta = sqrt((predicted_centre_x - true_centre_x)**2 + (predicted_centre_y - true_centre_y)**2)

    # Calculate Overlapping Area
    predicted_box = (top_left_corner[0], top_left_corner[1], bottom_right_corner[0], bottom_right_corner[1])
    true_box = (true_x, true_y, true_x + true_w - 1, true_y + true_h + 1)
    overlap_area = calculateOverlapArea(predicted_box, true_box)

    # Calculate Bounding Box Areas
    true_box_area = true_w * true_h
    predicted_box_area = (bottom_right_corner[0] - top_left_corner[0]) * (bottom_right_corner[1] - top_left_corner[1])
    percentage_error =  (abs(predicted_box_area - true_box_area) / abs(true_box_area)) * 100

    # Rounding
    centre_distance_delta = round(centre_distance_delta, 2)
    overlap_area = round(overlap_area, 2)

    # Add to dictionary
    centreDeltas[f"image{i:03d}"] = {
        "Euclidean Distance": centre_distance_delta,
        "Overlapping Area": overlap_area
    }
    overlappingAreas.append(overlap_area)

    # Determine Detection Pass rates
    if centre_distance_delta >= 250: failed += 1
    else: passed += 1

    cv2.imwrite(output_directory_path + f"/result{i:03d}.png", drawn_image)

with open(trainTestOutputPath, "w") as json_file:
    json.dump(centreDeltas, json_file, indent=4)

averageOverlappingArea = mean(overlappingAreas)
standardDeviation = stdev(overlappingAreas)

print(f"Passed: {passed}")
print(f"Failed: {failed}")
print(f"Passrate: {(passed / 500) * 100}%!")
print(f"Average Overlapping Area: {averageOverlappingArea}")
print(f"Standard Deviation: {standardDeviation}")





Passed: 398
Failed: 102
Passrate: 79.60000000000001%!
Average Overlapping Area: 52955.926
Standard Deviation: 58083.06462628575
